First, generate a jsonl file for the place files (with fields ID, document, and title)


In [ ]:
import os
import json
import pandas as pd

EXCEL_PATH    = 'protocols_place.xlsx'
SHEET_NAME    = 'new_data'
PLACE_FOLDER  = 'data/included_place_after_place_revision_csv'
OUTPUT_JSONL  = 'place_files_extracted.jsonl'

def main():
    # Read Excel, force ID and csv pairing to strings
    df = pd.read_excel(
        EXCEL_PATH,
        sheet_name=SHEET_NAME,
        dtype={'ID': str, 'csv pairing': str}
    )

    # Keep only rows with a non-empty csv pairing
    df = df[df['csv pairing'].notna() & (df['csv pairing'].str.strip() != '')]

    with open(OUTPUT_JSONL, 'w', encoding='utf-8') as out_f:
        for _, row in df.iterrows():
            trial_id = row['ID'].strip()
            csv_name = row['csv pairing'].strip()
            csv_path = os.path.join(PLACE_FOLDER, csv_name)

            if not os.path.isfile(csv_path):
                print(f'⚠️  File not found: {csv_path}')
                continue

            place_df = pd.read_csv(csv_path, dtype=str)

            if not {'Document', 'Title'}.issubset(place_df.columns):
                print(f'⚠️  Missing columns in {csv_name}: {place_df.columns.tolist()}')
                continue

            # Drop rows where BOTH document and title are empty/whitespace
            place_df['Document'] = place_df['Document'].fillna('')
            place_df['Title']    = place_df['Title'].fillna('')
            mask_nonempty = ~(
                (place_df['Document'].str.strip() == '') &
                (place_df['Title'].str.strip() == '')
            )
            place_df = place_df[mask_nonempty]

            # Build parallel lists
            documents = place_df['Document'].tolist()
            titles    = place_df['Title'].tolist()

            # Write one JSON object per CSV
            out_f.write(json.dumps({
                'id':        trial_id,
                'document': documents,
                'title':    titles
            }, ensure_ascii=False) + '\n')

    print(f'✅  Done! Wrote filtered JSONL to {OUTPUT_JSONL}')

if __name__ == '__main__':
    main()


✅  Done! Wrote filtered JSONL to place_files_extracted.jsonl


Now , change the copy of the extracted protocol text jsonl by replacing filename by trial number (ID)

In [ ]:
import json
import pandas as pd

EXCEL_PATH      = 'protocols_place.xlsx'
SHEET_NAME      = 'new_data'
INPUT_JSONL     = 'llm_extraction.jsonl'
OUTPUT_JSONL    = 'llm_extraction_with_ids.jsonl'

def build_title_to_id_map():
    df = pd.read_excel(
        EXCEL_PATH,
        sheet_name=SHEET_NAME,
        dtype={'Protocol title': str, 'ID': str, 'csv pairing': str}
    )
    df = df[df['csv pairing'].notna() & (df['csv pairing'].str.strip() != '')]
    # normalize to lowercase & strip
    df['Protocol title'] = df['Protocol title'].str.strip().str.lower()
    df['ID']             = df['ID'].str.strip()
    mapping = dict(zip(df['Protocol title'], df['ID']))
    print(f'✔️ Loaded {len(mapping)} title→ID mappings')
    return mapping

def remap_file_names():
    title_to_id = build_title_to_id_map()
    missing = set()

    with open(INPUT_JSONL, 'r', encoding='utf-8') as inp, \
         open(OUTPUT_JSONL, 'w', encoding='utf-8') as outp:

        for i, line in enumerate(inp, 1):
            record = json.loads(line)
            old_title = record.get('filename')
            if old_title is None:
                outp.write(json.dumps(record, ensure_ascii=False) + '\n')
                continue

            key = old_title.strip().lower()
            new_id = title_to_id.get(key)

            if new_id:
                record['filename'] = new_id
            else:
                missing.add(old_title)

            outp.write(json.dumps(record, ensure_ascii=False) + '\n')

    if missing:
        print(f'⚠️ Missing mapping for {len(missing)} titles:')
        for t in sorted(missing):
            print(f'    • "{t}"')

if __name__ == '__main__':
    remap_file_names()
    print(f'✅ Remapped JSONL written to {OUTPUT_JSONL}')


c:\Users\gabyf\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


✔️ Loaded 127 title→ID mappings
✅ Remapped JSONL written to llm_extraction_with_ids.jsonl


Finally, create a combined jsonl with id, extracted text, document, title by matching the two jsonl files by id (id in the protocols text is called filename)

In [2]:
import json

PROTOCOLS_JSONL = 'llm_extraction_with_ids.jsonl'    # has fields 'id' and 'text'
PLACES_JSONL    = 'place_files_extracted.jsonl'        # has fields 'id', 'documents', 'titles'
OUTPUT_JSONL    = 'llm_combined_data.jsonl'

def load_protocol_texts(path):
    """
    Build a map from trial-id (filename) → protocol text.
    Expects each line to have 'filename' and 'text'.
    """
    mapping = {}
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            rec = json.loads(line)
            pid = rec.get('filename')
            if pid is not None:
                mapping[pid] = rec.get('text', '')
    print(f'✔️ Loaded {len(mapping)} protocol texts')
    return mapping

def combine_per_id(proto_map, places_path, out_path):
    """
    For each record in the place-files JSONL, look up the protocol text
    and write a single merged record with:
      { id, text, document, title }
    """
    with open(places_path, 'r', encoding='utf-8') as inp, \
         open(out_path,      'w', encoding='utf-8') as outp:

        for line in inp:
            rec = json.loads(line)
            pid = rec.get('id')
            if pid is None:
                continue

            text = proto_map.get(pid)
            if text is None:
                # Skip if no matching protocol text
                continue

            doc   = rec.get('document', [])
            title = rec.get('title',    [])

            merged = {
                'id':        pid,
                'text':      text,
                'document': doc,
                'title':    title
            }
            outp.write(json.dumps(merged, ensure_ascii=False) + '\n')

    print(f'✅ Wrote {out_path}')

if __name__ == '__main__':
    protocol_map = load_protocol_texts(PROTOCOLS_JSONL)
    combine_per_id(protocol_map, PLACES_JSONL, OUTPUT_JSONL)


✔️ Loaded 127 protocol texts
✅ Wrote llm_combined_data.jsonl
